# DEEPX 튜토리얼 04 - DX‑STREAM 워크플로우

이 튜토리얼은 DX‑STREAM 개요와, 사용자가 만든 AI 모델을 DX‑STREAM 으로 실행하기 위한 end-to-end workflow 를 소개합니다.

학습 목표:
- DX‑STREAM 이 무엇인지 이해하고, DEEPX SDK 에서 어떻게 지원되는지 이해하기
- 핵심 Plugin 학습 (DxPreprocess → DxInfer → DxPostprocess → DxTracker → DxOsd)
- Custom AI 모델을 지원하기 위한 전/후처리 라이브러리 구현하기

>이 튜토리얼은 2025년 9월 공개된 dx‑all‑suite v2.0.0 을 기반으로 합니다.


## DX‑STREAM Overview
DEEPX NPU 에서 Vision AI 를 위한 GStreamer Plugin 들의 모음.
```text
Source → (decodebin) → DxPreprocess → DxInfer → DxPostprocess → DxTracker → DxOsd → Sink
```
- **DxPreprocess**: 리사이즈/정규화/레이아웃 변환 등 모델 입력에 맞는 전처리
- **DxInfer**: DX‑RT를 통해 NPU 추론 수행
- **DxPostprocess**: 모델 출력 해석 (예: YOLO 디텍션, 세그멘테이션 등)
- **DxTracker**: (선택) 다중 객체 추적
- **DxOsd**: 박스/라벨/마스크 등 오버레이

  ![img](assets/dx-stream-pipeline.png)

> 더 자세한 사항은 다음 DX-Stream User Guide 참고 👉 [here](https://developer.deepx.ai/?files=MjUxNw==)!


## 사전 준비 사항:

1. 아래 스크립트로 `dx‑stream` 경로로 이동 (루트 경로는 `ROOT_PATH` 환경변수값에 저장).

In [ ]:
# "dx-tutorials/dx-all-suite/dx-runtime/dx_stream" 디렉터리로 이동
import os
root_path = os.environ.get('ROOT_PATH')
%cd $root_path/dx-all-suite/dx-runtime/dx_stream

2. 다음 명령으로 사용될 **모델과 샘플 비디오**를 다운로드합니다:

In [ ]:
# Assets (모델/비디오)을 다운로드하여 assets/ 디렉터리에 배치합니다.
!./setup.sh

3. GStreamer에 DX‑STREAM 플러그인이 정상 등록되었는지 확인합니다: (아래 plugin 들이 설치되어야 함)
  - dxgather: DxGather
  - dxinfer: DXInfer
  - dxinputselector: DXInputSelector
  - dxmsgbroker: DXMsgBroker
  - dxmsgconv: DXMsgConv
  - dxosd: DXOsd
  - dxoutputselector: DXOutputSelector
  - dxpostprocess: DXPostprocess
  - dxpreprocess: DXPreprocess
  - dxrate: DXRate
  - dxtracker: DXTracker

In [ ]:
!gst-inspect-1.0 dxstream || echo 'dxstream plugin not found'

## DX_STREAM 일단 실행해 보기 (run_demo.sh)

### 1.Video 인풋으로 YOLOv7 pipeline 실행해 보기
   - `Note`: 상단의 stop button ('■') 을 눌러서 실행중인 cell 을 강제 종료 가능!

In [ ]:
# 입력 비디오 파일 경로
VIDEO_SRC='dx_stream/samples/videos/doughnut.mp4'

# GStreamer 파이프라인 설정
!gst-launch-1.0 filesrc location=$VIDEO_SRC ! decodebin ! \
                   dxpreprocess config-file-path=dx_stream/configs/Object_Detection/YoloV7/preprocess_config.json ! queue ! \
                   dxinfer config-file-path=dx_stream/configs/Object_Detection/YoloV7/inference_config.json ! queue ! \
                   dxpostprocess config-file-path=dx_stream/configs/Object_Detection/YoloV7/postprocess_config.json ! queue ! \
                   dxosd width=1280 height=720 ! queue ! \
                   videoconvert ! fpsdisplaysink sync=false

### 2. 각 DX plugin 들의 정의에 대해 **gst-inspect-1.0** 을 활용해 알아보기

In [ ]:
# Check dxpreprocess plugin
!gst-inspect-1.0 dxpreprocess
!echo "==============[ Configuration Start ]=============="
!cat dx_stream/configs/Object_Detection/YoloV7/preprocess_config.json
!echo "==============[  Configuration End  ]=============="

In [ ]:
# Check dxinfer plugin
!gst-inspect-1.0 dxinfer
!echo "==============[ Configuration Start ]=============="
!cat dx_stream/configs/Object_Detection/YoloV7/inference_config.json
!echo "==============[  Configuration End  ]=============="

In [ ]:
# Check dxpostprocess plugin
!gst-inspect-1.0 dxpostprocess
!echo "==============[ Configuration Start ]=============="
!cat dx_stream/configs/Object_Detection/YoloV7/postprocess_config.json
!echo "==============[  Configuration End  ]=============="

In [ ]:
# Check dxosd plugin
!gst-inspect-1.0 dxosd

## 커스텀 모델로 엔드‑투‑엔드 구성

아래는 예시로 **YOLOv7 Forklift + Person** 모델을 DX‑STREAM에서 구동하는 과정을 설명합니다.


In [ ]:
# (교육용 주석) Check the demo list - Total 8 GStreamer demos
!tail -n 15 run_demo.sh

### 1. 커스텀 후처리 라이브러리의 경로

후처리 로직은 `dx_stream/custom_library/postprocess_library` 아래에 모델별로 구현합니다. 예: `YoloV7/`

### 2. `dx_stream/custom_library/postprocess_library/YoloV7/postprocess.cpp` 수정:

모델 출력 텐서 포맷에 맞춰 디텍션 디코딩/스코어 임계값/클래스 매핑 등을 구현합니다.


In [ ]:
!./run_demo.sh <<< 0 # single_network/object_detectionS

In [ ]:
!./run_demo.sh <<< 1 # single_network/face_detection

In [ ]:
!./run_demo.sh <<< 3 # single_network/pose_estimation

In [ ]:
!./run_demo.sh <<< 4 # single_network/semantic_segmentation

> **힌트**: 기본 샘플을 복사해 시작하고, 출력 텐서의 shape/order를 먼저 확인하세요. 필요하면 NMS/스케일 복원/클래스 필터링을 추가합니다.


In [ ]:
!./run_demo.sh <<< 2 # tracking/run_YOLOV5S_tracker

### 3. 빌드 전 준비

필요한 헤더/링크 설정이 `CMakeLists.txt`에 반영되어 있는지 확인하세요.


In [ ]:
!./run_demo.sh <<< 5 # multi_stream/run_multi_stream_YOLOV5S

In [ ]:
!./run_demo.sh <<< 6 # rtsp/run_RTSP

## 커스텀 라이브러리 패치 적용 예시

아래 예시는 실습을 돕는 코드 수정 패치를 적용하는 방법을 보여줍니다.


In [ ]:
!./run_demo.sh <<< 7 # secondary_mode/run_secondary_mode

### 1. 패치 파일 위치

튜토리얼에 포함된 `patches/` 디렉터리의 패치를 사용합니다.


### 2. 패치가 변경하는 주요 내용

- YOLOv7 후처리 로직 보완
- 클래스 라벨/컬러매핑 정리
- 경계 박스 스케일 복원 로직 수정


> 변경 사항은 교육 목적의 예시이며, 실제 프로젝트에 맞춰 조정하세요.


In [ ]:
!wget "cs.deepx.ai/_deepx_fae_archive/dx-tutorials/yolov7-forklift-person.dxnn"

### 3. 패치 적용 방법

`git apply`로 수정사항을 반영합니다.


In [ ]:
DIFF_TEXT = r"""diff --git a/dx_stream/custom_library/postprocess_library/YoloV7/postprocess.cpp b/dx_stream/custom_library/postprocess_library/YoloV7/postprocess.cpp
index 7d048e9..26125ca 100644
--- a/dx_stream/custom_library/postprocess_library/YoloV7/postprocess.cpp
+++ b/dx_stream/custom_library/postprocess_library/YoloV7/postprocess.cpp
@@ -58,21 +58,11 @@ struct YoloConfig {
     float nms_threshold = 0.4f;      // IoU threshold for NMS

     // Number of classes in your dataset
-    int num_classes = 80;
+    int num_classes = 2;

     // COCO dataset class names (modify for your dataset)
     std::vector<std::string> class_names = {
-        "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck",
-        "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench",
-        "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra",
-        "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee",
-        "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove",
-        "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
-        "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange",
-        "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch",
-        "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse",
-        "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
-        "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
+        "Forklift", "Worker"
     };
 };

"""

# (교육용 주석) 파일로 저장
with open("dx_stream_update.diff", "w", encoding="utf-8") as f:
    f.write(DIFF_TEXT)

print("Saved: dx_stream_update.diff (bytes)", len(DIFF_TEXT))

### 3. `git apply` 명령으로 수정 코드 적용:

In [ ]:
!git apply --whitespace=fix dx_stream_update.diff

### 4. DX‑STREAM 라이브러리 재빌드:

In [ ]:
!source ../venv-dx-runtime/bin/activate && ./build.sh

### 5. 커스텀 모델(yolov7‑forklift‑person.dxnn)을 불러올 **구성 JSON** 작성

In [ ]:
import json, os
yolov7_custom = {
    "preprocess_id": 1,
    "inference_id": 1,
    "model_path" : "./yolov7-forklift-person.dxnn"
}
with open("yolov7-forklift-person.json", "w") as f: json.dump(yolov7_custom, f, indent=2)

### 6. 테스트용 비디오 복사

In [ ]:
!cp ../../../assets/forklift-worker.mp4 ./

### 7. 아래 DX‑STREAM 파이프라인 실행

In [ ]:
# 입력 비디오 파일 경로
VIDEO_SRC='forklift-worker.mp4'

# GStreamer 파이프라인 설정
!gst-launch-1.0 filesrc location=$VIDEO_SRC ! decodebin ! \
                   dxpreprocess config-file-path=dx_stream/configs/Object_Detection/YoloV7/preprocess_config.json ! queue ! \
                   dxinfer config-file-path=yolov7-forklift-person.json ! queue ! \
                   dxpostprocess config-file-path=dx_stream/configs/Object_Detection/YoloV7/postprocess_config.json ! queue ! \
                   dxosd width=1280 height=720 ! queue ! \
                   videoconvert ! fpsdisplaysink sync=false

## 디버그 팁

GStreamer 파이프라인 구성을 이미지로 덤프하기:

In [ ]:
!sudo apt install -y graphviz

In [ ]:
!export GST_DEBUG_DUMP_DOT_DIR=./ && ./run_demo.sh <<< 0

In [ ]:
# (교육용 주석) !dot -Tpng -o NULL_READY.png *NULL_READY.dot
# (교육용 주석) !dot -Tpng -o READY_PAUSED.png *READY_PAUSED.dot
!dot -Tpng -o PAUSED_PLAYING.png *PAUSED_PLAYING.dot
# (교육용 주석) !dot -Tpng -o PLAYING_PAUSED.png *PLAYING_PAUSED.dot
# -Tpng : 출력 형식을 png로 지정
# -o : 출력 파일 이름
# *.dot : 입력 파일 이름


In [ ]:
from IPython.display import Image, display

# 생성된 이미지를 표시
display(Image(filename="PAUSED_PLAYING.png"))